## EDA

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving marketing_data.csv to marketing_data.csv


In [ ]:
import pandas as pd

# Load the dataset
file_path = 'marketing_data.csv'
data = pd.read_csv(file_path)

# Display basic information about the dataset
data_info = {
    "Head": data.head(),
    "Shape": data.shape,
    "Data Types": data.dtypes,
    "Missing Values": data.isnull().sum(),
    "Duplicate Rows": data.duplicated().sum()
}

data_info


{'Head':     age           job  marital  education default  balance housing loan  \
 0  58.0    management  married   tertiary      no     2143     yes   no   
 1  44.0    technician   single  secondary      no       29     yes   no   
 2  33.0  entrepreneur  married  secondary      no        2     yes  yes   
 3  47.0   blue-collar  married    unknown      no     1506     yes   no   
 4  33.0       unknown   single    unknown      no        1      no   no   
 
    contact  day month  campaign  pdays  previous Location poutcome   y  
 0  unknown    5   may         1     -1         0   Naples  unknown  no  
 1  unknown    5   may         1     -1         0  Firenze  unknown  no  
 2  unknown    5   may         1     -1         0   Milano  unknown  no  
 3  unknown    5   may         1     -1         0   Naples  unknown  no  
 4  unknown    5   may         1     -1         0   Napoli  unknown  no  ,
 'Shape': (43097, 17),
 'Data Types': age          float64
 job           object
 marital

In [ ]:
# Step 1: Handle Missing Data
# Impute 'age' with median, 'contact' and 'poutcome' with mode
data['age'].fillna(data['age'].median(), inplace=True)
data['contact'].fillna(data['contact'].mode()[0], inplace=True)
data['poutcome'].fillna(data['poutcome'].mode()[0], inplace=True)

# Step 2: Remove Duplicates
data.drop_duplicates(inplace=True)

# Step 3: Outlier Detection and Treatment for numeric features using IQR method
numeric_columns = ['age', 'balance', 'campaign', 'pdays', 'previous']

for col in numeric_columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Capping outliers
    data[col] = data[col].clip(lower=lower_bound, upper=upper_bound)

# Step 4: Encode Categorical Columns
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan',
                       'contact', 'month', 'Location', 'poutcome', 'y']

# One-hot encode nominal categorical variables
data_encoded = pd.get_dummies(data, columns=categorical_columns[:-1], drop_first=True)

# Encode target variable ('y') as binary (1 for 'yes', 0 for 'no')
data_encoded['y'] = data['y'].map({'yes': 1, 'no': 0})

# Step 5: Standardize Numeric Features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

data_encoded[numeric_columns] = scaler.fit_transform(data_encoded[numeric_columns])

# Check final dataset
final_data_info = {
    "Description": ["Shape After Cleaning", "Missing Values After Cleaning"],
    "Value": [
        str(data_encoded.shape),  # Convert shape to string
        data_encoded.isnull().sum().sum(),  # Total missing values
        ]
}

# Convert the dictionary to a DataFrame
final_data_df = pd.DataFrame(final_data_info)
final_data_df


In [ ]:
data_encoded['y'].value_counts()

,count
y,
0,39918
1,3172
